In [ ]:
import pandas as pd
from pathlib import Path
import xarray as xr
from datetime import timedelta
import os

pad = Path(os.getcwd())
parent = pad.parent
path = parent/'output_caravan'/'timeseries'/'netcdf'/'vlaamsebekken'/'6.nc'
df = xr.open_dataset(path)
df_df = df.to_dataframe()


df_filtered = df_df[['potential_evaporation_sum_FAO_PENMAN_MONTEITH', 'total_precipitation_sum','streamflow']]
df_filtered = df_filtered.reset_index(drop = False)
df_filtered = df_filtered[df_filtered['date'] >= '1972-06-30']
df_filtered = df_filtered.rename(columns={'date': 'Timestamp', 'potential_evaporation_sum_FAO_PENMAN_MONTEITH': 'potential_evaporation_sum'})

print(df_filtered)

In [13]:

area = 111_639_970.52
# Ensure Timestamp is in datetime format
df_filtered["Timestamp"] = pd.to_datetime(df_filtered["Timestamp"])
df_filtered['streamflow'] = (df_filtered['streamflow']*area)/1000/3600/24
# Keep only Timestamp and streamflow
df_q = df_filtered[["Timestamp", "streamflow"]].copy()

# Define path to save
path_new = Path(parent / 'data/Zwalm_data/output_Q/Q_data.pkl')

# Save as pickle
df_q.to_pickle(path_new)


In [5]:
df_filtered["Timestamp"] = pd.to_datetime(df_filtered["Timestamp"])

# Expand each daily row into 24 hourly rows
hourly_data = []
for _, row in df_filtered.iterrows():
    for hour in range(24):
        new_row = row.copy()
        new_row["Timestamp"] = row["Timestamp"] + timedelta(hours=hour)
        new_row["potential_evaporation_sum"] /= 24  # Divide by 24
        new_row["total_precipitation_sum"] /= 24  # Divide by 24
        hourly_data.append(new_row)
 


In [ ]:
df_hourly = pd.DataFrame(hourly_data)
print(df_hourly)

In [32]:
output_path = Path('data/Zwalm_data/preprocess_output/Final_Forcings_PDM.pkl')
df_hourly.to_pickle(output_path)